![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

In [1]:
!pip install --upgrade chromadb==0.3.26 pydantic==1.10.0 sentence-transformers


In [17]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
import getpass
import pandas as pd
import re


def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [3]:
model_id = "meta-llama/llama-3-70b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [4]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 4096,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [5]:
project_id = "4f8122aa-25c0-437a-bd85-8f0c9de1b6fc"
#space_id = os.getenv("SPACE_ID")


In [6]:
credentials = get_credentials()
print(credentials)  # Pour vérifier ce que retourne cette fonction


{'url': 'https://eu-de.ml.cloud.ibm.com', 'apikey': 'krqI2LsLocdB5PJzcJhRwQJ9Pi97bO_js3HWyD4GtASu'}


In [7]:
!pip install ibm-watsonx-ai
#!pip install --force-reinstall ibm-watsonx-ai


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [8]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	#space_id = space_id
	)


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [9]:
from ibm_watsonx_ai.client import APIClient

wml_credentials = get_credentials()
client = APIClient(credentials=wml_credentials, project_id=project_id)

vector_index_id = "0f98afe1-1994-4312-93c0-777883083a29"
vector_index_details = client.data_assets.get_details(vector_index_id)
vector_index_properties = vector_index_details["entity"]["vector_index"]

from ibm_watsonx_ai.foundation_models.embeddings.sentence_transformer_embeddings import SentenceTransformerEmbeddings

emb = SentenceTransformerEmbeddings('sentence-transformers/all-MiniLM-L6-v2')

import subprocess
import gzip
import json
import chromadb
import random
import string

def hydrate_chromadb():
    data = client.data_assets.get_content(vector_index_id)
    content = gzip.decompress(data)
    stringified_vectors = str(content, "utf-8")
    vectors = json.loads(stringified_vectors)

    chroma_client = chromadb.Client()

    # make sure collection is empty if it already existed
    collection_name = "my_collection"
    try:
        collection = chroma_client.delete_collection(name=collection_name)
    except:
        print("Collection didn't exist - nothing to do.")
    collection = chroma_client.create_collection(name=collection_name)

    vector_embeddings = []
    vector_documents = []
    vector_metadatas = []
    vector_ids = []

    for vector in vectors:
        vector_embeddings.append(vector["embedding"])
        vector_documents.append(vector["content"])
        metadata = vector["metadata"]
        lines = metadata["loc"]["lines"]
        clean_metadata = {}
        clean_metadata["asset_id"] = metadata["asset_id"]
        clean_metadata["asset_name"] = metadata["asset_name"]
        clean_metadata["url"] = metadata["url"]
        clean_metadata["from"] = lines["from"]
        clean_metadata["to"] = lines["to"]
        vector_metadatas.append(clean_metadata)
        asset_id = vector["metadata"]["asset_id"]
        random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        id = "{}:{}-{}-{}".format(asset_id, lines["from"], lines["to"], random_string)
        vector_ids.append(id)

    collection.add(
        embeddings=vector_embeddings,
        documents=vector_documents,
        metadatas=vector_metadatas,
        ids=vector_ids
    )
    return collection

chroma_collection = hydrate_chromadb()

def proximity_search( question ):
    query_vectors = emb.embed_query(question)
    query_result = chroma_collection.query(
        query_embeddings=query_vectors,
        n_results=vector_index_properties["settings"]["top_k"],
        include=["documents", "metadatas", "distances"]
    )

    documents = list(reversed(query_result["documents"][0]))

    return "\n".join(documents)


c:\Users\YacineKHITER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Collection didn't exist - nothing to do.


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [10]:
prompt_input = """<|start_header_id|>system<|end_header_id|>

You always answer the questions with markdown formatting. The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. You must omit that you answer the questions with markdown.

Any HTML tags must be wrapped in block quotes, for example ```<html>```. You will be penalized for not rendering code in block quotes.

When returning code blocks, specify language.

Given the document and the current conversation between a user and an assistant, your task is as follows: answer any user query by using information from the document. Always answer as helpfully as possible, while being safe. When the question cannot be answered using the context or document, output the following response: "I cannot answer that question based on the provided document.".

Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|eot_id|><|start_header_id|>user<|end_header_id|>

__grounding__<|eot_id|><|start_header_id|>user<|end_header_id|>

genere moi des cas de test qui prennent en compte toutes les user stories suivantes en tenant compte du format du document en vecteur index joint:

Voici les user stories



ID UC-002
Name IBM Order Validation
Description 1. As a customer, I want my server order to be validated by IBM to know if it is accepted, rejected, or modified so that I can proceed accordingly.

The user should be able to submit a server order to IBM and receive a response on the order validation status. The possible validation statuses are:
                                       OK: The order is accepted as is.
                                       Not OK: The order is rejected.
                                       Modified: The order is modified by IBM.
The user should be able to see the validation details in a summary table with the following information:

                                   Validation status (OK, Not OK, Modified)
                                   Modification details (if applicable)
                                   Comments from the IBM team



For each status, the user should be able to perform the following actions:

                                 OK: The user can confirm the order and proceed with the payment.
                                 Not OK: The user can see the reasons for the rejection and submit a new order.
         Modified: The user can see the details of the modifications made by IBM and accept or reject these modifications.



The user should be able to receive an email notification about the order validation status with a link to access the details.

The user should be able to export the validation details to a csv/xl file.


================================================================================

Extracted text from IBM server order UC-001.docx:



ID UC-001
Name IBM Server Order
Description 1. As a customer, I want to be able to order a server from IBM with a specific configuration and a detailed price so that it meets my business needs.

The user should be able to configure the server by choosing from the following options:

                                  Processor type (Intel, AMD)
                                 Amount of RAM (16GB, 32GB, 64GB, etc.)
                                 Storage capacity (1TB, 2TB, SSD, HDD, etc.)
                                 Operating system (Linux, Windows Server, etc.)
                                 Network options (Gigabit Ethernet, Fibre Channel, etc.)
The user should be able to see a summary of the total price based on the selected options.
Clicking on each option should allow viewing the details and prices of each configuration:

                               Processor – Clicking on this option should display the available processor types with their respective prices.

                       RAM – Clicking on this option should display the different available RAM capacities with their respective prices.
   Storage – Clicking on this option should display the available storage types and capacities with their respective prices.



The user should be able to see the details of the chosen configuration in a summary table with the following information:

        Processor type
       Amount of RAM
       Storage capacity
       Operating system
       Network options
Total price



The user should be able to view a bar graph representing the cost of each component (processor, RAM, storage, etc.) and the total cost. Clicking on the graph should display additional details on the cost of each component over a 14-day period. The user should be able to export the configuration and price to a csv/xl file. The user should be able to expand and view the graph in full screen.




et voici le template des résultats, les noms au début sont les noms des colonnes:

Sr .No Requirement ID Test Case ID Test Case Title Test Case Description Test Steps Expected Results Test Data
1 RT_01 TC_1 Live_Dashboard_View_Test Live Dashboard View Testing: Verify that the Squad Leader and Manager can view the Live Dashboard to get the latest updates on their team members' statuses. "1. Access the ABC Secured Credit Card (Platinum Mastercard) login page.

Enter valid login credentials (username and password) for a customer with a ABC Secured Credit Card (Platinum Mastercard).

Click the ""Log in"" button.

Verify that the system logs in the customer successfully.

Navigate to the Credit Card Account Details Page

Click on the ""Accounts"" or ""Credit Cards"" tab (if applicable).
Select the ABC Secured Credit Card (Platinum Mastercard) account from the list of available accounts.
Verify Account Details Accuracy

Check that the credit limit, balance, and payment due date are displayed correctly.
Verify that the information matches the customer's records or expectations.
Test Account Details Page Navigation

Click on different sections of the account details page (e.g., ""Account Activity,"" ""Payment History,"" ""Rewards"").
Verify that the system navigates to the corresponding pages without errors.
Verify Account Management Options

Check that the account details page includes options for managing the account, such as ""Make a Payment"" or ""Request Credit Limit Increase.""
Verify that these options are functional and lead to the appropriate pages.
Verify Permissions and Authorization

Ensure that the system validates the customer's permissions and authorization to access their account details.
Test this by attempting to access the account details page with an invalid username or password.
Logout

Click the ""Logout"" or ""Sign Out"" button to end the session.
Verify Logout Success

Confirm that the system logs the customer out successfully.
Check that the login page is displayed again, indicating a successful logout." "1. Access the ABC Secured Credit Card (Platinum Mastercard) login page:
Successfully navigate to the login page of the ABC Secured Credit Card (Platinum Mastercard) account.
The navigation should be intuitive and error-free.
Enter valid login credentials (username and password) for a customer with a ABC Secured Credit Card (Platinum Mastercard):

Entering valid login credentials should allow the customer to log in to their account without errors.
Click the ""Log in"" button:

Clicking the ""Log in"" button should initiate the login process and direct the customer to their account details page.
Verify that the system logs in the customer successfully:

The system should log in the customer successfully, displaying their account details page.<|eot_id|><|start_header_id|>assistant<|end_header_id|><|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here are the test cases for the user stories:

**UC-001: IBM Server Order**

| Sr. No | Requirement ID | Test Case ID | Test Case Title | Test Case Description | Test Steps | Expected Results | Test Data |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | UC-001 | TC_1 | Server Configuration Test | Verify that the user can configure the server with specific options. | 1. Access the server order page.<br>2. Select processor type (Intel, AMD).<br>3. Select amount of RAM (16GB, 32GB, 64GB, etc.).<br>4. Select storage capacity (1TB, 2TB, SSD, HDD, etc.).<br>5. Select operating system (Linux, Windows Server, etc.).<br>6. Select network options (Gigabit Ethernet, Fibre Channel, etc.). | The user should be able to configure the server with the selected options. | Processor type: Intel, AMD<br>Ram: 16GB, 32GB, 64GB<br>Storage: 1TB, 2TB, SSD, HDD<br>Operating System: Linux, Windows Server<br>Network Options: Gigabit Ethernet, Fibre Channel |
| 2 | UC-001 | TC_2 | Summary Table Test | Verify that the user can see a summary table with the selected configuration. | 1. Access the server order page.<br>2. Configure the server with specific options.<br>3. Click on the "Summary" button. | The summary table should display the selected configuration, including processor type, amount of RAM, storage capacity, operating system, and network options. | Same as above |
| 3 | UC-001 | TC_3 | Bar Graph Test | Verify that the user can view a bar graph representing the cost of each component. | 1. Access the server order page.<br>2. Configure the server with specific options.<br>3. Click on the "Bar Graph" button. | The bar graph should display the cost of each component, including processor, RAM, storage, and network options. | Same as above |
| 4 | UC-001 | TC_4 | Export Configuration Test | Verify that the user can export the configuration and price to a csv/xl file. | 1. Access the server order page.<br>2. Configure the server with specific options.<br>3. Click on the "Export" button. | The user should be able to export the configuration and price to a csv/xl file. | Same as above |

**UC-002: IBM Order Validation**

| Sr. No | Requirement ID | Test Case ID | Test Case Title | Test Case Description | Test Steps | Expected Results | Test Data |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | UC-002 | TC_1 | Order Submission Test | Verify that the user can submit a server order to IBM. | 1. Access the server order page.<br>2. Configure the server with specific options.<br>3. Click on the "Submit" button. | The user should be able to submit the server order to IBM. | Same as above |
| 2 | UC-002 | TC_2 | Order Validation Status Test | Verify that the user can receive a response on the order validation status. | 1. Access the server order page.<br>2. Submit the server order to IBM.<br>3. Wait for the order validation response. | The user should receive a response on the order validation status, including OK, Not OK, or Modified. | Order validation status: OK, Not OK, Modified |
| 3 | UC-002 | TC_3 | Validation Details Test | Verify that the user can see the validation details in a summary table. | 1. Access the server order page.<br>2. Submit the server order to IBM.<br>3. Wait for the order validation response.<br>4. Click on the "Validation Details" button. | The summary table should display the validation details, including validation status, modification details, and comments from the IBM team. | Same as above |
| 4 | UC-002 | TC_4 | Email Notification"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [25]:
question = input("Question: ")
grounding = proximity_search(question)
formattedQuestion = f"""<|begin_of_text|><|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"AI: {generated_response}")


# Regular expression to parse the table rows
pattern = re.compile(r"\|\s(\d+)\s\|\s([\w\-]+)\s\|\s([\w\-]+)\s\|\s(.+?)\s\|\s(.+?)\s\|\s(.+?)\s\|\s(.+?)\s\|\s(.+?)\s\|")

matches = pattern.findall(generated_response)

# Create a DataFrame from the parsed data
columns = ["Sr. No", "Requirement ID", "Test Case ID", "Test Case Title", "Test Case Description", "Test Steps", "Expected Results", "Test Data"]
df = pd.DataFrame(matches, columns=columns)

# Clean up the 'Test Steps' and 'Expected Results' fields by replacing '<br>' with newline
df['Test Steps'] = df['Test Steps'].str.replace('<br>', '\n')
df['Expected Results'] = df['Expected Results'].str.replace('<br>', '\n')

# Save the DataFrame to an Excel file
output_file_path = 'C:\\Users\\YacineKHITER\\cross-user-stories-test-case-generation\\test_cases.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Test cases have been exported to {output_file_path}")

AI: Here are the test cases for the user stories:

**UC-001: IBM Server Order**

| Sr. No | Requirement ID | Test Case ID | Test Case Title | Test Case Description | Test Steps | Expected Results | Test Data |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | UC-001 | TC_1 | Server Configuration Test | Verify that the user can configure the server with specific options. | 1. Access the server order page.<br>2. Select processor type (Intel, AMD).<br>3. Select amount of RAM (16GB, 32GB, 64GB, etc.).<br>4. Select storage capacity (1TB, 2TB, SSD, HDD, etc.).<br>5. Select operating system (Linux, Windows Server, etc.).<br>6. Select network options (Gigabit Ethernet, Fibre Channel, etc.). | The user should be able to configure the server with the selected options. | Processor type: Intel, AMD<br>Ram: 16GB, 32GB, 64GB<br>Storage: 1TB, 2TB, SSD, HDD<br>Operating System: Linux, Windows Server<br>Network Options: Gigabit Ethernet, Fibre Channel |
| 2 | UC-001 | TC_2 | Summary Table Test | 

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  